# Week 3 Assignment: Segmenting and Clustering in the City of Toronto, Canada
## Huanglei Pan

## Part 1 :  Scraping and parsing the data

In [1]:
# Import the packages
import pandas as pd

### 1-1 Scrape the Dataframe from Wikipedia Page and create pandas dataframe

In [2]:
# Read the table from Wikipedia
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

# Convert the table to pandas dataframe
df= pd.DataFrame(table)
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### 1-2 Deal with missing data and aggregate data

In [3]:
#Drop the rows with Borughs that are not assigned
df=df[df.Borough != 'Not assigned']
df.head(12)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [4]:
# Replace the not assigned neighbourhoods by their borough names
df['Neighbourhood']=df['Neighbourhood'].replace('Not assigned', df['Borough'])
df.head(12)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [5]:
#Aggregate the neighbourhoods with some borough
df_toronto=df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ','.join(x))
df_toronto.reset_index(level=['Postcode','Borough'], inplace=True)
df_toronto.head(12)
        

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### 1-3 Check the shape of dataframe

In [6]:
# See the shape of the final dataframe
print("There are " + str(df_toronto.shape[0]) + " rows in the dataframe")

There are 103 rows in the dataframe


## Part 2 :  Get the Coordinates for the Postcodes

In [7]:
# Install and import geocoder package
!conda install -c conda-forge geocoder --yes
import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

In [8]:
# Get the coordinates of the postcodes
latitude=[]
longitude=[]
for postcode in df_toronto.Postcode:
    lat_lng_coords = None

    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postcode)) #geocoder.google is unavailble, so I use geocoder.arcgis
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

In [9]:
# Add the coordinates to the dataframe
df_toronto['Latitude'] = latitude
df_toronto['Longitude'] = longitude
df_toronto.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723270,-79.451286
4,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715
5,M9A,Queen's Park,Queen's Park,43.662299,-79.528195
6,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
7,M3B,North York,Don Mills North,43.749055,-79.362227
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.707535,-79.311773
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180


## Part 3 :  Segmenting and Clustering the Neibourhoods

In [10]:
# Import pakages
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import numpy as np

In [11]:
# Install and import folium libary
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 1

### 3-1 Mapping Toronto Neighbourhoods

In [12]:
#Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# Add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3-2 Explore Neighbourhoods in Toronto by Foursquare

In [117]:
# The code was removed by Watson Studio for sharing.

In [16]:
# Create a function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# Fit the function on data_toronto
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights,Lawrence Manor
Queen's Park
Queen's Park
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design Exchange,Toronto 

In [18]:
# Check the size of results
print(toronto_venues.shape)
toronto_venues.head()

(1105, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752420,-79.329242,Brookbanks Park,43.751976,-79.332140,Park
1,Victoria Village,43.730600,-79.313265,Wigmore Park,43.731023,-79.310771,Park
2,Harbourfront,43.650295,-79.359166,The Distillery Historic District,43.650244,-79.359323,Historic Site
3,Harbourfront,43.650295,-79.359166,Arvo,43.649963,-79.361442,Coffee Shop
4,Harbourfront,43.650295,-79.359166,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop


In [19]:
# Check how many venues were returned for each neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Agincourt,2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",11,11,11,11,11,11
"Alderwood,Long Branch",4,4,4,4,4,4
"Bathurst Manor,Downsview North,Wilson Heights",3,3,3,3,3,3
Bayview Village,2,2,2,2,2,2
"Bedford Park,Lawrence Manor East",13,13,13,13,13,13
Berczy Park,7,7,7,7,7,7
"Birch Cliff,Cliffside West",4,4,4,4,4,4


In [78]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 218 uniques categories.


### 3-3 Analyze Each Neighborhood

In [79]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,...,Trail,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
toronto_onehot.shape

(1105, 219)

In [81]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,...,Trail,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.0,0.040000,0.00,0.00,0.00,0.060,0.0,0.00,...,0.0,0.0,0.020000,0.00,0.000000,0.000000,0.02,0.000000,0.0,0.00
1,Agincourt,0.000000,0.0,0.000000,0.00,0.00,0.00,0.000,0.0,0.00,...,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00
2,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.000000,0.00,0.00,0.00,0.000,0.0,0.00,...,0.0,0.0,0.000000,0.00,0.090909,0.000000,0.00,0.000000,0.0,0.00
3,"Alderwood,Long Branch",0.000000,0.0,0.000000,0.00,0.00,0.00,0.000,0.0,0.25,...,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00
4,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.000000,0.00,0.00,0.00,0.000,0.0,0.00,...,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00
5,Bayview Village,0.000000,0.0,0.000000,0.00,0.00,0.00,0.000,0.0,0.00,...,0.5,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00
6,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.00,0.00,0.00,0.000,0.0,0.00,...,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00
7,Berczy Park,0.000000,0.0,0.000000,0.00,0.00,0.00,0.000,0.0,0.00,...,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00
8,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.00,0.00,0.00,0.000,0.0,0.00,...,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00
9,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.000000,0.0,0.000000,0.00,0.00,0.00,0.000,0.0,0.00,...,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00


In [82]:
toronto_grouped.shape

(85, 219)

In [83]:
# Create function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [84]:
# Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Asian Restaurant,Steakhouse,Café,Bar,Gastropub,Seafood Restaurant,Gym,American Restaurant,Pizza Place
1,Agincourt,Skating Rink,Badminton Court,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Hardware Store,Japanese Restaurant,Pharmacy,Caribbean Restaurant,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,Beer Store,Pizza Place,Grocery Store
3,"Alderwood,Long Branch",Construction & Landscaping,Performing Arts Venue,Athletics & Sports,Burmese Restaurant,Dance Studio,Deli / Bodega,Flower Shop,Fish Market,Fish & Chips Shop,Field
4,"Bathurst Manor,Downsview North,Wilson Heights",Men's Store,Photography Studio,Nightclub,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


### 3-4 Clustering Neighborhoods

In [86]:
# Import Kmeans libary
from sklearn.cluster import KMeans

Using KMeans to cluster the neigborhoods

In [87]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:20]

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [88]:
# Add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto


# Aerge toronto_grouped with data_toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,1.0,Park,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M4A,North York,Victoria Village,43.730600,-79.313265,1.0,Park,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,0.0,Bakery,Art Gallery,Café,Coffee Shop,Antique Shop,Restaurant,Mexican Restaurant,Spanish Restaurant,Japanese Restaurant,Italian Restaurant
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723270,-79.451286,0.0,Clothing Store,Coffee Shop,Women's Store,Bakery,Toy / Game Store,Men's Store,Fast Food Restaurant,Leather Goods Store,Restaurant,Electronics Store
4,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715,0.0,Coffee Shop,Baseball Field,College Auditorium,Café,Fried Chicken Joint,Sandwich Place,Salad Place,Bar,Bank,Metro Station
5,M9A,Queen's Park,Queen's Park,43.662299,-79.528195,0.0,Coffee Shop,Baseball Field,College Auditorium,Café,Fried Chicken Joint,Sandwich Place,Salad Place,Bar,Bank,Metro Station
6,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,M3B,North York,Don Mills North,43.749055,-79.362227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.707535,-79.311773,0.0,Pizza Place,Bus Line,Spa,Pharmacy,Pet Store,Fast Food Restaurant,Café,Gastropub,Intersection,Bank
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180,0.0,Coffee Shop,Café,Movie Theater,Middle Eastern Restaurant,Sandwich Place,College Rec Center,Ramen Restaurant,Japanese Restaurant,Burger Joint,Lake


In [89]:
toronto_merged.shape

(103, 16)

In [90]:
# Drop rows with NaN
toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)

In [91]:
toronto_merged.reset_index(drop=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int32')

In [92]:
toronto_merged.shape

(86, 16)

In [93]:
toronto_merged.reset_index(drop=True)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,1,Park,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M4A,North York,Victoria Village,43.730600,-79.313265,1,Park,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,0,Bakery,Art Gallery,Café,Coffee Shop,Antique Shop,Restaurant,Mexican Restaurant,Spanish Restaurant,Japanese Restaurant,Italian Restaurant
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723270,-79.451286,0,Clothing Store,Coffee Shop,Women's Store,Bakery,Toy / Game Store,Men's Store,Fast Food Restaurant,Leather Goods Store,Restaurant,Electronics Store
4,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715,0,Coffee Shop,Baseball Field,College Auditorium,Café,Fried Chicken Joint,Sandwich Place,Salad Place,Bar,Bank,Metro Station
5,M9A,Queen's Park,Queen's Park,43.662299,-79.528195,0,Coffee Shop,Baseball Field,College Auditorium,Café,Fried Chicken Joint,Sandwich Place,Salad Place,Bar,Bank,Metro Station
6,M4B,East York,"Woodbine Gardens,Parkview Hill",43.707535,-79.311773,0,Pizza Place,Bus Line,Spa,Pharmacy,Pet Store,Fast Food Restaurant,Café,Gastropub,Intersection,Bank
7,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180,0,Coffee Shop,Café,Movie Theater,Middle Eastern Restaurant,Sandwich Place,College Rec Center,Ramen Restaurant,Japanese Restaurant,Burger Joint,Lake
8,M6B,North York,Glencairn,43.707990,-79.448367,0,Astrologer,Yoga Studio,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
9,M4C,East York,Woodbine Heights,43.689640,-79.306874,0,Playground,Pet Store,Coffee Shop,Donut Shop,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### 3-5 Visualize the resulting clusters

In [94]:
# Import Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3-6 Observations for the clustering result

In [113]:
# Counting the number for each cluster
stat = toronto_merged.groupby('Cluster Labels').count()
stat = stat.iloc[:,:1]
stat.rename(columns = {"Postcode": "Number"}, inplace=True)
stat.reset_index()

,Cluster Labels,Number
0,0,75
1,1,5
2,2,1
3,3,3
4,4,2


In [97]:
# Seperate each cluster results to see their common venues
createVar = locals()
for i in range(5):    
    createVar['Cluster_'+str(i)] =  toronto_merged[toronto_merged['Cluster Labels']==i]

In [98]:
Cluster_0

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,0,Bakery,Art Gallery,Café,Coffee Shop,Antique Shop,Restaurant,Mexican Restaurant,Spanish Restaurant,Japanese Restaurant,Italian Restaurant
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723270,-79.451286,0,Clothing Store,Coffee Shop,Women's Store,Bakery,Toy / Game Store,Men's Store,Fast Food Restaurant,Leather Goods Store,Restaurant,Electronics Store
4,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715,0,Coffee Shop,Baseball Field,College Auditorium,Café,Fried Chicken Joint,Sandwich Place,Salad Place,Bar,Bank,Metro Station
5,M9A,Queen's Park,Queen's Park,43.662299,-79.528195,0,Coffee Shop,Baseball Field,College Auditorium,Café,Fried Chicken Joint,Sandwich Place,Salad Place,Bar,Bank,Metro Station
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.707535,-79.311773,0,Pizza Place,Bus Line,Spa,Pharmacy,Pet Store,Fast Food Restaurant,Café,Gastropub,Intersection,Bank
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180,0,Coffee Shop,Café,Movie Theater,Middle Eastern Restaurant,Sandwich Place,College Rec Center,Ramen Restaurant,Japanese Restaurant,Burger Joint,Lake
10,M6B,North York,Glencairn,43.707990,-79.448367,0,Astrologer,Yoga Studio,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
14,M4C,East York,Woodbine Heights,43.689640,-79.306874,0,Playground,Pet Store,Coffee Shop,Donut Shop,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
15,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481,0,Restaurant,Coffee Shop,Gastropub,Hotel,BBQ Joint,Bakery,Movie Theater,Breakfast Spot,Performing Arts Venue,Church
16,M6C,York,Humewood-Cedarvale,43.692105,-79.430355,0,Hockey Arena,Field,Tennis Court,Dog Run,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [99]:
Cluster_1

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,1,Park,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M4A,North York,Victoria Village,43.730600,-79.313265,1,Park,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
64,M9N,York,Weston,43.704845,-79.517546,1,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
69,M6P,West Toronto,"High Park,The Junction South",43.659935,-79.463019,1,Park,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
72,M2R,North York,Willowdale West,43.777695,-79.445797,1,Park,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [100]:
Cluster_2

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713054,-79.285055,2,Soccer Field,Yoga Studio,Donut Shop,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [101]:
Cluster_3

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193,3,Construction & Landscaping,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
39,M2K,North York,Bayview Village,43.781015,-79.380529,3,Construction & Landscaping,Trail,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
50,M9L,North York,Humber Summit,43.759500,-79.557028,3,Construction & Landscaping,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [102]:
Cluster_4

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,M4T,Central Toronto,"Moore Park,Summerhill East",43.690685,-79.382946,4,Tennis Court,Yoga Studio,Food,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
91,M4W,Downtown Toronto,Rosedale,43.682205,-79.377945,4,Playground,Tennis Court,Food,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### Report

From the clustering result we can find:

1. Cluster 0 contains 75/86 (87.2%) of the total boroughs, these neigbourhoods have common venues that are related to food and drinks, and almost contain Café, coffee shop and restaurants in top 5 common venues.
     
2. There are 5 neighbourhoods in Cluser 1, all of them have Yoga Studio in top 3 common venues and four of they have  Yoga Studio, Donut Shop and Fish Market in top 3 common venues and 

3. Cluster 2 has only 1 neighbourhoods, it's hard to summarize its characteristics.

4. Cluster 3 has 3 neighbourhoods, all of them have Construction & Landscaping in the first common venue,  and have same categories in 4th to 10th common venues.

5. Cluster 4 has 2 neighbourhoods, both of them have venues related to sports in the first two places, and have have same categories in 3th to 10th common venues.
